# Parte 7

Aqui foi criado o aplicativo para o Modelo 2.

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from jupyter_dash import JupyterDash
from datetime import datetime as dt
from datetime import timedelta
from dash.dependencies import Input, Output
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [2]:
df_1 = pd.read_csv('Demanda\Demanda_2019.csv', sep=';', header=None, names=['Horário','Data', 'Demanda', 'Unidade'],skiprows=5,
                parse_dates=True) 
df_2 = pd.read_csv('Demanda\Demanda_2020.csv', sep=';', header=None, names=['Horário','Data', 'Demanda', 'Unidade'],skiprows=5,
                parse_dates=True) 
df= pd.concat([df_1, df_2],ignore_index=True)
df.head()

,Horário,Data,Demanda,Unidade
0,00:00:00,01/01/2019 00:00:00 (hv),"3.479,00",MW F
1,00:01:00,01/01/2019 00:01:00 (hv),"3.471,00",MW F
2,00:02:00,01/01/2019 00:02:00 (hv),"3.468,00",MW F
3,00:03:00,01/01/2019 00:03:00 (hv),"3.466,00",MW F
4,00:04:00,01/01/2019 00:04:00 (hv),"3.457,00",MW F


In [3]:
df['Demanda'] = df['Demanda'].str[:-3]
df['Demanda'] = pd.to_numeric(df['Demanda'], errors='coerce')
df = df.drop(columns=['Unidade','Horário'])
df['Data'] = df['Data'].str[:16]
df['Data'] = pd.to_datetime(df['Data'], format = '%d/%m/%Y %H:%M', errors='coerce')

In [4]:
df_new = df.resample('60T', on='Data').mean()
df_new.index.freq='H'
df_new['Demanda']=df_new['Demanda']*1000

In [5]:
app = JupyterDash()

In [6]:
app.layout = html.Div(style={'textAlign': 'center', 'width': '1300px', 'font-family': 'Verdana'},
                      children=[
    html.H2(children='Previsão da Demanda de Energia Elétrica no RS', style={'color': 'rgb(25,25,112)'}),     
       
    html.Div(["Data Inicial: ",
                          
    dcc.DatePickerSingle(
        id='my-date-picker-single',
        min_date_allowed=dt(2019, 1, 1, 0, 0),
        max_date_allowed=dt(2020, 8, 23, 0, 0),
        initial_visible_month=dt(2019, 1, 1, 0, 0),
        date=str(dt(2020, 8, 23, 0, 0, 0))
    ),

             ], style={'display': 'inline-block'}), 

    html.Div(id='output-container-date-picker-single'),
    
    dcc.Graph(id='graphic')
])

In [7]:
@app.callback(
    Output('graphic', 'figure'),
    [Input('my-date-picker-single', 'date'),
     ])
def update_graph(date):        
          
    train_data  = df_new.loc[:date]
    train_data  = train_data.iloc[-672:]
    test_data = df_new.loc[date:]
    test_data = test_data.iloc[24:192]
    
    model = ExponentialSmoothing(train_data['Demanda'],
                                    trend='add',
                                    seasonal = 'mul',
                                    seasonal_periods=24*7)
    model = model.fit(smoothing_seasonal=0.4, smoothing_level=0.6)
    test_predictions = model.forecast(len(test_data))          

    fig = px.line(y=round(train_data.iloc[-504:]['Demanda'],0), x=train_data.iloc[-504:].index, labels={'x':'', 'y':'Demanda'})     
    #fig.add_scatter(y=test_data['Demanda'],x=test_data.index, mode='lines', name='Teste')
    fig.add_scatter(y=round(test_predictions,0), x=test_data.index, mode='lines', name='Previsão')     
    
    return fig

In [8]:
app.run_server(mode='external', port=8040)

Dash app running on http://127.0.0.1:8040/


C:\Users\camil\Anaconda3\envs\TC Pos\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\camil\Anaconda3\envs\TC Pos\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

